In [1]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators

In [35]:
# Generate SMIRNOFF ligand mol object
mol = oechem.OEMol()
molpdb = '../OpenMMCubes/tests/input/toluene.pdb'
ffxml = open('../OpenMMCubes/tests/input/forcefield/smirff99Frosst.ffxml', 'r')
with oechem.oemolistream(molpdb) as ifs:
    oechem.OEReadMolecule(ifs, mol)
    
    # Generate molecule System, encode to bytes to attach generic data.
    from smarty.forcefield import ForceField
    mol_ff = ForceField(ffxml)
    mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
    print("System is: ",type(mol_sys))
    encoded_sys = openmm.XmlSerializer.serialize(mol_sys).encode()
    print("Encoded system is:: ",type(encoded_sys))
    
    # Generate parmed Structure
    molecule_structure = parmed.openmm.load_topology(mol_top, mol_sys, xyz=mol_pos)
    molecule_structure.residues[0].name = "MOL"

    # Serialize to pickle object for bytes representation.
    import pickle
    pkl_struct = pickle.dumps(molecule_structure)
    print("Structure is: ",type(molecule_structure))
    print("Pickled structure is: ",type(pkl_struct))
    
    pkl_top = pickle.dumps(mol_top)
    print("Topology is: ", type(mol_top))
    print("Pickled top is: ", type(pkl_top))

    
    with oechem.oemolostream('smirff_mol.oeb.gz') as ofs:
        mol.AddData(oechem.OEGetTag('system'), encoded_sys)
        mol.AddData(oechem.OEGetTag('structure'), pkl_struct)
        mol.AddData(oechem.OEGetTag('topology'), pkl_top)
        # System and Structure are both in bytes, but pkl does not attach
        # Try attaching as string?
        mol.AddData(oechem.OEGetTag('structure_str'), str(pkl_struct))
        oechem.OEWriteMolecule(ofs, mol)
        
    ffxml.close()

System is:  <class 'simtk.openmm.openmm.System'>
Encoded system is::  <class 'bytes'>
Structure is:  <class 'parmed.structure.Structure'>
Pickled structure is:  <class 'bytes'>
Topology is:  <class 'simtk.openmm.app.topology.Topology'>
Pickled top is:  <class 'bytes'>


In [36]:
datamol = oechem.OEMol()
with oechem.oemolistream('smirff_mol.oeb.gz') as ifs:
    oechem.OEReadMolecule(ifs, datamol)
    print(datamol.GetData().keys())
    
    if 'structure_str' in datamol.GetData().keys():
        struct_str = datamol.GetData(oechem.OEGetTag('structure_str'))
        #print(struct_str.encode())
        # Turn back to bytes?
        pickle.loads(struct_str.encode())

dict_keys(['system', 'structure_str'])


UnpicklingError: unpickling stack underflow